# Changement de variable sur simplex

In [1]:
#install.packages("SphericalCubature")
library("SphericalCubature")
help("rect2polar")

Warning message:
"package 'SphericalCubature' was built under R version 4.0.5"
Loading required package: cubature

Warning message:
"package 'cubature' was built under R version 4.0.5"
Loading required package: SimplicialCubature

Loading required package: mvmesh

Warning message:
"package 'mvmesh' was built under R version 4.0.5"
Loading required package: rcdd

Warning message:
"package 'rcdd' was built under R version 4.0.5"
If you want correct answers, use rational arithmetic.
See the Warnings sections in help pages for
    functions that do computational geometry.

Loading required package: rgl

Warning message:
"package 'rgl' was built under R version 4.0.5"
Loading required package: geometry

Warning message:
"package 'geometry' was built under R version 4.0.5"
Loading required package: abind



rect2polar          package:SphericalCubature          R Documentation

_n-_d_i_m_e_n_s_i_o_n_a_l _p_o_l_a_r _c_o_o_r_d_i_n_a_t_e _t_r_a_n_s_f_o_r_m_a_t_i_o_n_s

_D_e_s_c_r_i_p_t_i_o_n:

     Convert between polar and rectangular coordinates in n-dimensions.
     The point (x[1],...,x[n]) in rectangular coordinates corresponds
     to the point (r,phi[1],...,phi[n-1]) in polar coordinates.

_U_s_a_g_e:

     polar2rect(r, phi)
     rect2polar(x)
     
_A_r_g_u_m_e_n_t_s:

       r: a vector of radii of length m.

     phi: angles, a matrix of size (n-1) x m.

       x: (n x m) matrix, with column j being the point in
          n-dimensional space.

_D_e_t_a_i_l_s:

     n dimensional polar coordinates are given by the following:
     rectangular x=(x[1],...,x[n]) corresponds to polar
     (r,phi[1],...,phi[n-1]) by
     x[1] = r*cos(phi[1])
     x[j] = r*prod(sin(phi[1:(j-1)]))*cos(phi[j]), 2 <= j <= n-1
     ...

In [2]:
data<-read.csv("data_visco.csv")
data_lasso <- read.csv("data_lasso.csv")
data_AIC <- read.csv("data_AIC.csv")

### Change of  variables from a simplex to a N-sphere with the N-sphere coordinates

In [3]:
change_in_phi_from_x = function(x){ #We put in argument a matrix or a vector that takes n values in a simplex
    epsilon<-1e-9  # We had a little noise to insure that we can devide by 0
    X<-as.matrix(x[, -1]/100)
    X[which(X == NaN)] <- 0.
    for (i in 1:dim(X)[1]){   #To ensure that we each row is equal to 1 to remove the noise in the column with the highest values in it
        X[ i,23] <- X[ i,23] - (sum(X[i, ]==0.))*epsilon
    }
    X[which(X == 0.)] <- epsilon
    Y<-sqrt(X) #We do a first change of variables y²=x eq y=sqrt(x)
    Y[is.na(Y)] <- 0  #If we have some Nan in our matrix we replace them by zeros
    return(t(as.matrix(rect2polar(t(Y))$phi)))
}



### Change of  variables from a N-sphere to an simplex with the N-sphere coordinates

In [9]:
change_in_x_from_phi = function(phi){ #We put in argumennt a matrix or a vector that takes n-1 values of angles in [0, pi/2]
    Phi<-as.matrix(phi)  
    r=rep(1,dim(Phi)[1])  #To realize our change of variables considered in the function polar2rect in R it needs to take a radius equals to 1 
    new<-(t(polar2rect(r, t(Phi))))**2  # We realize the change of variables such that f.e. x_1=cos²(theta_1)
    new[which(new<1e-4)]<-0   #We obtain sometimes values of order 1e-5 we replace it by 0
    return (new)
}


### Change of  variables from an simplex to an hypercube with the length ratio variables

In [10]:
change_in_lambda_from_x = function(x){  # We put in argument a matrix or a vector that takes n values in a simplex
    X<-as.matrix(x)
    Phi<-change_in_phi_from_x(X) #We change our variables from the simplex in angles in [0, pi/2 ]
    lambda<-(cos(Phi))**2 # We change of varaible is the following : lambda=cos(theta)
    return(lambda)
}

### Change of  variables from an hypercube to a simplex which the length ratio variables

In [11]:
change_in_x_from_lambda = function(lambda){ #We put in argument a matrix or a vector of n-1 variables in an hypercube
    Lambda<-as.matrix(lambda) 
    Phi<-acos(sqrt(Lambda)) #We change our variables from the hypercube to the N-spheres so n-1 angles in [0, pi/2]
    return(change_in_x_from_phi(Phi)) #We use the change of variables from a N-sphere to a simplex
}

## TESTS OF THE FUNCTIONS

### Test of the change of variables for the N-spheres coordinates

In [12]:
print(dim(data_lasso[, -1]))
print(data_lasso[1,-1 ]/100)
print(change_in_phi_from_x(data_lasso[1, ]))
print(change_in_x_from_phi(change_in_phi_from_x(data_lasso[1, ])))
print(sum(change_in_x_from_phi(change_in_phi_from_x(data_lasso[2, ]))))

[1] 7247   28
  X__3   X__4   X__8 X__10 X__17 X__18 X__29 X__30 X__43 X__48 X__49 X__53
1    0 0.0942 0.0852     0     0     0     0     0     0     0     0     0
  X__57 X__59 X__64 X__67 X__71  X__74 X__84 X__86 X__106 X__109 X__113 X__115
1     0     0     0     0     0 0.1367     0     0      0 0.6839      0      0
  X__117 X__133 X__138 autres
1      0      0      0      0


Warning message in sqrt(X):
"NaNs produced"


         [,1]     [,2]    [,3]     [,4]     [,5]     [,6]     [,7]     [,8]
[1,] 1.570765 1.258841 1.25908 1.570761 1.570761 1.570761 1.570761 1.570761
         [,9]    [,10]    [,11]    [,12]    [,13]    [,14]    [,15]    [,16]
[1,] 1.570761 1.570761 1.570761 1.570761 1.570761 1.570761 1.570761 1.570761
        [,17]    [,18]    [,19]    [,20]    [,21]        [,22]    [,23]
[1,] 1.570761 1.150371 1.570758 1.570758 1.570758 8.550445e-05 1.570796
        [,24]    [,25]     [,26]     [,27]
[1,] 1.107149 1.047198 0.9553166 0.7853982


Warning message in sqrt(X):
"NaNs produced"


     [,1]   [,2]   [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13]
[1,]    0 0.0942 0.0852    0    0    0    0    0    0     0     0     0     0
     [,14] [,15] [,16] [,17]  [,18] [,19] [,20] [,21]  [,22] [,23] [,24] [,25]
[1,]     0     0     0     0 0.1367     0     0     0 0.6839     0     0     0
     [,26] [,27] [,28]
[1,]     0     0     0


Warning message in sqrt(X):
"NaNs produced"


[1] 1


### Test of the change of variables for the length ratio

In [13]:
print(data_lasso[1:2,-1 ]/100)
print(change_in_lambda_from_x(data_lasso[1:2, ]))
print(change_in_x_from_lambda(change_in_lambda_from_x(data_lasso[1:2, ])))
print(sum(change_in_x_from_lambda(change_in_lambda_from_x(data_lasso[1:2, ]))))                             

  X__3   X__4   X__8 X__10 X__17 X__18 X__29 X__30 X__43 X__48 X__49 X__53
1    0 0.0942 0.0852     0     0     0     0     0     0     0     0     0
2    0 0.0942 0.0853     0     0     0     0     0     0     0     0     0
  X__57 X__59 X__64 X__67 X__71  X__74 X__84 X__86 X__106 X__109 X__113 X__115
1     0     0     0     0     0 0.1367     0     0      0 0.6839      0      0
2     0     0     0     0     0 0.1173     0     0      0 0.7032      0      0
  X__117 X__133 X__138 autres
1      0      0      0      0
2      0      0      0      0


Warning message in sqrt(X):
"NaNs produced"


      [,1]   [,2]      [,3]         [,4]         [,5]         [,6]         [,7]
[1,] 1e-09 0.0942 0.0940605 1.218620e-09 1.218620e-09 1.218620e-09 1.218620e-09
[2,] 1e-09 0.0942 0.0941709 1.218769e-09 1.218769e-09 1.218769e-09 1.218769e-09
             [,8]         [,9]        [,10]        [,11]        [,12]
[1,] 1.218620e-09 1.218620e-09 1.218620e-09 1.218620e-09 1.218621e-09
[2,] 1.218769e-09 1.218769e-09 1.218769e-09 1.218769e-09 1.218769e-09
            [,13]        [,14]        [,15]        [,16]        [,17]     [,18]
[1,] 1.218621e-09 1.218621e-09 1.218621e-09 1.218621e-09 1.218621e-09 0.1665854
[2,] 1.218769e-09 1.218769e-09 1.218769e-09 1.218769e-09 1.218769e-09 0.1429616
            [,19]        [,20]        [,21] [,22]        [,23] [,24] [,25]
[1,] 1.462202e-09 1.462202e-09 1.462202e-09     1 3.749152e-33   0.2  0.25
[2,] 1.422071e-09 1.422071e-09 1.422071e-09     1 3.749152e-33   0.2  0.25
         [,26] [,27]
[1,] 0.3333333   0.5
[2,] 0.3333333   0.5


Warning message in sqrt(X):
"NaNs produced"


     [,1]   [,2]   [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13]
[1,]    0 0.0942 0.0852    0    0    0    0    0    0     0     0     0     0
[2,]    0 0.0942 0.0853    0    0    0    0    0    0     0     0     0     0
     [,14] [,15] [,16] [,17]  [,18] [,19] [,20] [,21]  [,22] [,23] [,24] [,25]
[1,]     0     0     0     0 0.1367     0     0     0 0.6839     0     0     0
[2,]     0     0     0     0 0.1173     0     0     0 0.7032     0     0     0
     [,26] [,27] [,28]
[1,]     0     0     0
[2,]     0     0     0


Warning message in sqrt(X):
"NaNs produced"


[1] 2
